# Deskriptive Analyse
Eine deskriptive Analyse des gesamten annotierten Datensets. (Paragraphen) Mit Fokus auf:
1. regionale vs. nationale Zeitungen
2. Ressorts
3. Anzahl der Paragraphen im Zeitverlauf
4. Länge der Paragraphen
5. Annotationen

Allgemeine Daten:
- Alle Artikel: 60'866'588
- Alle gefilterten Artikel ~25'000
- Alle annotierten Paragraphen: 1'268
- Alle annotierten Paragraphen mit relevantem Label: 101

### Import

In [ ]:
# packages
import os
import re
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm 
from datetime import datetime
from ast import literal_eval
import plotly.graph_objects as go
import plotly.express as px
import json


In [ ]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# change cwd
%cd drive/MyDrive/Work/Frontline/data
#%cd /content/drive/MyDrive/data/

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [ ]:
from scripts import annotations

In [ ]:
data_all=pd.read_csv("elinor/annotation_test_05_18.csv")

In [ ]:
# annotated data
dfs=[]
for doc in os.listdir("annotated"):
  if doc.startswith("annotations"):
    #read json data
    json_data=json.load(open("annotated/"+doc, encoding="utf-8"))
    #convert to dataframe
    data=pd.DataFrame(json_data["documents"])
    #for now: filter out paragraphs that have not been annotated 
    data=data[data["annotations"].apply(len)>0]   
    data.loc[:,"file"]=doc 
    dfs.append(data)

     

# merge jsons
data=pd.concat(dfs)
data=data.reset_index(drop=True)

### Daten Vorbereiten

In [ ]:
# extract name,date,ressort information
mask=data.file!="annotations_50sample_06_09.json"
data.loc[mask,"name"]=data[mask].attributes_flat.apply(lambda x: x["name"])
data.loc[mask,"ressort"]=data[mask].attributes_flat.apply(lambda x: x["ressort"])
data.loc[mask,"datum"]=data[mask].attributes_flat.apply(lambda x: x["datum"])
mask=data.duplicated("text",False)
data.loc[mask,"name"]=data.loc[mask].sort_values(["text","file"])["name"].fillna(method="ffill",inplace=False)
data.loc[mask,"ressort"]=data.loc[mask].sort_values(["text","file"])["ressort"].fillna(method="ffill",inplace=False)
data.loc[mask,"datum"]=data.loc[mask].sort_values(["text","file"])["datum"].fillna(method="ffill",inplace=False)

In [ ]:
mask=data.file!="annotations_50sample_06_09.json"
data.loc[mask,"id"]=data[mask].attributes_flat.apply(lambda x: x["artikel_id"])
data.loc[mask,"ressort"]=data[["id"]].loc[mask].merge(data_all[["artikel_id","ressort"]], left_on="id", right_on="artikel_id",how="left")["ressort"]
data.loc[mask,"datum"]=data[["id"]].loc[mask].merge(data_all[["artikel_id","datum"]], left_on="id", right_on="artikel_id",how="left")["datum"]

In [ ]:
# merge duplicated annotations
data=data.sort_values("file").drop_duplicates("text",keep="last")

In [ ]:
# extract annotations
data.loc[:,"annotations"]=data.loc[:,"annotations"].apply(annotations.extract_annotations)

In [ ]:
data.loc[:,"jaccard"]=data.loc[:,"annotations"].apply(annotations.calculate_similarity)
data.loc[:,"dice"]=data.loc[:,"annotations"].apply(annotations.calculate_similarity,args=["dice"])

In [ ]:
data.loc[:,"annotations"]=data.apply(annotations.ground_truth_filter,min_coannotation=1,min_similarity=-1, similarity="dice",axis=1)

In [ ]:
def clean_annotation(ann):
  # removes the label "domestic violence" if it has been chosen in combination with a second label
  if len(ann)>1:
    ann.discard("Domestic Violence")
  return ann


In [ ]:
#remove domestic violence label if chosen in combination with another label
data.annotations=data.annotations.apply(clean_annotation)

In [ ]:
# extract annotation with label != Domestic Violence
data=data[data.annotations!={"Domestic Violence"}]

### Constants

In [ ]:
regional_newspapers=['Aachener Nachrichten',
 'Aachener Zeitung',
 'Alb Bote',
 'Aller-Zeitung',
 'Allgemeine Zeitung Mainz-Rheinhessen',
 'Badische Zeitung',
 'Barmstedter Zeitung',
 'Bayerische Rundschau',
 'Bergedorfer Zeitung',
 'Bergische Morgenpost',
 'Berliner Kurier',
 'Berliner Morgenpost',
 'Berliner Zeitung',
 'Bersenbrücker Kreisblatt',
 'Bonner General-Anzeiger',
 'Bote vom Haßgau',
 'Bramscher Nachrichten',
 'Braunschweiger Zeitung',
 'Coburger Tageblatt',
 'DIE KITZINGER',
 'Darmstädter Echo',
 'Der Prignitzer',
 'Dresden am Wochenende',
 'Dresdner Morgenpost',
 'Dresdner Neueste Nachrichten',
 'Döbelner Allgemeine Zeitung',
 'Eichsfelder Tageblatt',
 'Ems-Zeitung',
 'Freie Presse',
 'Fränkischer Tag',
 'Gießener Anzeiger',
 'Göttinger Tageblatt',
 'Haller Tagblatt',
 'Hamburger Abendblatt',
 'Hamburger Morgenpost',
 'HarzKurier',
 'Heilbronner Stimme',
 'Hessische Niedersächsische Allgemeine',
 'Hildesheimer Allgemeine Zeitung',
 'Hochheimer Zeitung',
 'Hofheimer Zeitung',
 'Hohenloher Tagblatt',
 'Hohenzollerische Zeitung',
 'Husumer Nachrichten',
 'Höchster Kreisblatt',
 'Idsteiner Zeitung',
 'Kieler Nachrichten',
 'Kirner Zeitung',
 'Kreis-Anzeiger',
 'Lampertheimer Zeitung',
 'Landshuter Zeitung',
 'Lausitzer Rundschau - Elbe-Elster-Rundschau',
 'Lauterbacher Anzeiger',
 'Le Monde Diplomatique',
 'Leipziger Volkszeitung',
 'Lingener Tagespost',
 'Lübecker Nachrichten',
 'Magdeburger General-Anzeiger','Münsterland Zeitung',
 'Nahe Zeitung', 
 'Nassauische Neue Presse',
 'Neue Osnabrücker Zeitung','Neue Ruhr/Rhein Zeitung',
 'Neue Westfälische',
 'Neue Württembergische Zeitung',
 'Neuss-Grevenbroicher Zeitung',
 'Neue Presse Hannover',
 'Magdeburger Volksstimme','Main-Post',
 'Main-Spitze','Nordbayerischer Kurier',
 'Norddeutsche Neueste Nachrichten',
 'Norddeutsche Rundschau','Nordkurier',
 'Nordwest-Zeitung',
 'Nürnberger Nachrichten','Oberhessische Zeitung',
 'Oeffentlicher Anzeiger',
 'Oschatzer Allgemeine Zeitung',
 'Osterländer Volkszeitung','Passauer Neue Presse',
 'Peiner Allgemeine Zeitung',
 'Reutlinger General-Anzeiger',
 'Rhein-Hunsrück-Zeitung',
 'Rhein-Lahn-Zeitung',
 'Rhein-Zeitung','Ruhr Nachrichten',
 'Rundschau für den Schwäbischen Wald',
 'Rüsselsheimer Echo',
 'Saale-Zeitung',
 'Saarbrücker Zeitung',
 'Salzgitter-Zeitung','Thüringische Landeszeitung',
 'Trierischer Volksfreund',
 'Usinger Anzeiger',
 'Volksblatt Würzburg',
 'Volkszeitung Schweinfurt'
 'Ostsee-Zeitung',
 'Ostthüringer Zeitung','Schweinfurter Tagblatt',
 'Schweriner Volkszeitung','Ostsee-Zeitung',
 'Volkszeitung Schweinfurt',
 'Westerwälder Zeitung',
 'Wiesbadener Kurier',
 'Wiesbadener Tagblatt',
 'Schwäbische Zeitung',
 'Segeberger Zeitung',
 'Solinger Morgenpost','Nürnberger Zeitung',
 'Straubinger Tagblatt',
 'Sächsische Zeitung',
 'Südkurier',
 'Taunus Zeitung','Wittlager Kreisblatt',
 'Wolfenbütteler Zeitung',
 'Wolfsburger Allgemeine Zeitung',
 'Wolfsburger Nachrichten',
 'Wormser Zeitung',
 'Thüringer Allgemeine',
 'Straubinger Tagblatt'
 'Nürnberger Zeitung',
 'Kölnische Rundschau']
national_newspapers=['Audio Video Foto Bild','Börsen-Zeitung','F.A.Z. Einspruch',
 'F.A.Z. Wirtschaftswissenschaft','FOCUS',
 'Frankfurter Neue Presse','Handelsblatt',
 'Handelsblatt Morning Briefing',"Jüdische Allgemeine","Süddeutsche Zeitung",
 'Frankfurter Rundschau',"Hannoversche Allgemeine Zeitung", 'ZEIT Campus',
 'ZEIT Studienführer',
 'ZEIT Wissen','Kölner Stadt-Anzeiger',
 'taz.die tageszeitung',"LiteraturSPIEGEL",
 'Christ und Welt','B.Z.',"BILD","BUSINESS INSIDER DEUTSCHLAND",'Spiegel Start',
 'Sport Bild', 'WELT KOMPAKT',
 'WELT ONLINE','Meller Kreisblatt',
 'Meppener Tagespost',
 'Metzinger Uracher Volksblatt',
 'Mittelbayerische Zeitung','Westfalen-Blatt',
 'Westfalenpost',
 'Mitteldeutsche Zeitung',"Wirtschaftszeitung",
 'WELT am SONNTAG',
 'Stern', "Märkische Allgemeine","SÜDWEST PRESSE",'Westdeutsche Allgemeine Zeitung',
 'Westdeutsche Zeitung','Le Monde Diplomatique',"Westfälische Rundschau","Rheinische Post",'Münchner Merkur',"BILD am Sonntag","Abendzeitung",'DIE ZEIT online','ZEIT Geschichte','DIE ZEIT', "DER SPIEGEL","DIE WELT","EXPRESS","taz","SPIEGEL Plus",'Bild der Frau','DER SPIEGEL online']


### 1. Regional vs. National

In [ ]:
data.loc[:,"category"]=["regional" if name in regional_newspapers else "national" for name in data.name ]

<ipython-input-17-5a39e86ec590>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"category"]=["regional" if name in regional_newspapers else "national" for name in data.name ]


In [ ]:
fig=px.pie(names=list(data["category"].value_counts().keys()), title='Artikel zu Häuslicher Gewalt nach Kategorie: national/ regional',values=data["category"].value_counts())
fig.show()

In [ ]:
print("Anzahl der vorkommenden Zeitungen: ",data.name.nunique())
top=20
print(f"\nTop {top} der vorkommenden Zeitungen:\n")
round(data.name.value_counts(normalize=True)[:top].mul(100),2).astype(str)+" %"

Anzahl der vorkommenden Zeitungen:  41

Top 20 der vorkommenden Zeitungen:



BILD                               17.82 %
WELT ONLINE                        14.85 %
Hildesheimer Allgemeine Zeitung     4.95 %
B.Z.                                3.96 %
Ruhr Nachrichten                    3.96 %
Schwäbische Zeitung                 2.97 %
Neue Presse Hannover                2.97 %
Rhein-Hunsrück-Zeitung              2.97 %
Westfalenpost                       2.97 %
Coburger Tageblatt                  2.97 %
Neue Ruhr/Rhein Zeitung             2.97 %
Wolfsburger Allgemeine Zeitung      2.97 %
DIE WELT                            1.98 %
Berliner Morgenpost                 1.98 %
BILD am Sonntag                     1.98 %
Berliner Kurier                     1.98 %
Westdeutsche Allgemeine Zeitung     1.98 %
Fränkischer Tag                     0.99 %
Wiesbadener Tagblatt                0.99 %
Hamburger Abendblatt                0.99 %
Name: name, dtype: object

### 2. Ressorts

In [ ]:
data.ressort=data.ressort.str.title()
ressort_rename={"PANORAMA":"Panorama","Titel":"Titelseite","Meinung Und Debatte":"Debatte","Aus Der Region":"Region",
                "Ausland":"Aus Aller Welt","Blick In Die Welt":"Aus Aller Welt","Antworten Und Debatte":"Debatte",
                "/Wn/Regiodesk Wb/Aus Aller Welt":"Aus Aller Welt","Rund Um Die Welt":"Aus Aller Welt", "Welt":"Aus Aller Welt","Weltspiegel":"Aus Aller Welt",
                "Welt Im Spiegel":"Aus Aller Welt",
                "Welt Aktuell":"Aus Aller Welt",
                "Deutschland":"Inland",
                "Land":"Inland",
                "F.A.Z. Einspruch":"Justiz",
                "Nrw Und Deutschland":"Inland"
}
data.ressort=data.ressort.replace(ressort_rename)


In [ ]:
ressort_rename_regex={"[A-z +\&/\n,0-9;]*Lokal\S*[&äA-z \/]*":"Lokales",
                      "Ressort:\s":"",
                      "\S*[A-z \/\n]*Politik\S*[A-z \/\n]*":"Politik",
                      "\S*[A-z \&/\n]*Kultur\S*[A-z \/]*":"Kultur",
                      "Titel\S*[A-z \/]*":"Titelseite","\S*[A-z \&/\n]*Justiz\S*[&äA-z \/]*":"Justiz",
                      "\S*[A-z \&/\n]*Hintergrund\S*[&äA-z \/]*":"Hintergrund",
                      "\S*[A-z \&/\n]*Leser\S*[&äA-z \/]*":"Leser",
                      "\S*[A-z \&/\n]*Medien\S*[&äA-z \/]*":"Medien",
                      "\S*[A-z \&/\n]*Forum\S*[&äA-z \/]*":"Forum",
                      "[A-z +\&/\n,0-9\-;]*Meinung\S*[&äA-z \/]*":"Meinung",
                      "[A-z +\&/\n,0-9()Üß\-;]*Sport\S*[&äA-z \/]*":"Sport",
                      "[A-z ]*Aktuell\S*[&äA-z \/]*":"Aktuell",
                      "[A-z &ü\-]*Region\S*[\&\-äA-z \/]*":"Region",
                      "[A-z &ü\-]*region\S*[\&\-äA-z \/]*":"Region",
                      "[A-z &ü\-]*Panorama\S*[\&\-äA-z \/]*":"Panorama",
        
    
}
data.ressort=data.ressort.replace(ressort_rename_regex,regex=True)

In [ ]:
data_ressort=data[["ressort"]]
data_ressort=data_ressort[data_ressort.ressort!=""]

In [ ]:
print("Anzahl der vorkommenden Ressorts: ",data.ressort.nunique())

Anzahl der vorkommenden Ressorts:  20


In [ ]:
fig = px.pie(names=data_ressort.ressort.value_counts(normalize=True).keys(), title='Artikel zu Häuslicher Gewalt nach Ressort\n',values=list(data_ressort.ressort.value_counts(normalize=True)))
fig.show()

### 3. Anzahl der Artikel im Zeitverlauf

In [ ]:
data.datum=data.datum.apply(pd.to_datetime,format="%Y%m%d")

In [ ]:
data_grouped=data.groupby(["datum"]).count()["id"]
data_grouped=pd.DataFrame(data_grouped).reset_index().rename(columns={"id":"Veröffentlichte Artikel (pro Tag)", "datum":"Datum"})

In [ ]:
data_grouped_monthly=data_grouped
data_grouped_monthly.Datum=[i +pd.offsets.MonthBegin(-1) for i in data_grouped_monthly.Datum]
data_grouped_monthly=data_grouped_monthly.groupby("Datum").sum().reset_index().rename(columns={"Veröffentlichte Artikel (pro Tag)":"Veröffentlichte Artikel (pro Monat)"})
fig = px.bar(data_grouped_monthly, y= "Veröffentlichte Artikel (pro Monat)",x="Datum", title='Veröffentlichte Artikel zum Thema Häusliche Gewalt',)
fig.show()

In [ ]:
data_grouped_res=data
data_grouped_res.datum=[i +pd.offsets.MonthBegin(-1) for i in data_grouped_res.datum]
data_grouped_res=data_grouped_res.groupby(["datum","ressort"]).count()[["id"]]
data_grouped_res=pd.DataFrame(data_grouped_res).reset_index().rename(columns={"id":"Veröffentlichte Artikel (pro Monat)", "datum":"Datum","ressort":"Ressort"})
fig = px.bar(data_grouped_res, y= "Veröffentlichte Artikel (pro Monat)",x="Datum", title='Veröffentlichte Artikel zum Thema Häusliche Gewalt',color="Ressort")
fig.show()

In [ ]:
data_grouped_cat=data
data_grouped_cat.datum=[i +pd.offsets.MonthBegin(-1) for i in data_grouped_cat.datum]
data_grouped_cat=data_grouped_cat.groupby(["datum","category"]).count()[["id"]]
data_grouped_cat=pd.DataFrame(data_grouped_cat).reset_index().rename(columns={"id":"Veröffentlichte Artikel (pro Monat)", "datum":"Datum","category":"Kategorie"})
fig = px.bar(data_grouped_cat, y= "Veröffentlichte Artikel (pro Monat)",x="Datum", title='Veröffentlichte Artikel zum Thema Häusliche Gewalt',color="Kategorie")
fig.show()

### 4. Länge von Paragraphen
-> beachte: Paragrphen mit Überlänge (>120 Wörter) wurden rausgefiltert

In [ ]:
data["lengths"]=[len(i.split()) for i in data.text]

In [ ]:
fig=px.histogram(data.rename(columns={"category":"Kategorie"}),"lengths", nbins=50, title="Länge von Paragraphen zum Thema häusliche Gewalt", color="Kategorie")
fig.update_layout(yaxis_title="Anzahl der Artikel")
fig.update_layout(xaxis_title="Anzahl der Wörter eines Paragraphen")
fig.show()

In [ ]:
fig=px.bar(data.rename(columns={"lengths":"Länge"}).groupby("ressort")["Länge"].mean(),title="Länge der Paragraphen nach Ressort")
fig.update_layout(yaxis_title="Länge der Paragraphen (Wörter)")
fig.update_layout(xaxis_title="Ressorts")
fig.show()

## 5. Annotationen

In [ ]:
names=[list(i) for i in data.annotations.value_counts().keys()]

In [ ]:
fig = px.pie(names=names, title='Labels der Annotierten Paragraphen',values=list(data.annotations.value_counts()))
fig.show()

In [ ]:
data["annotations"]=[", ".join(list(ann)) for ann in data.annotations]

In [ ]:
px.bar(data.groupby(["file","annotations"]).count()[["id"]].reset_index(),x="file",y="id",color="annotations")